In [ ]:
import chipwhisperer as cw

In [ ]:
bitstream_path = "C:\\Users\\jhaaa\\Desktop\\cw305_research\\vivado\\neo430\\neo430.runs\\impl_1\\cw305_top.bit"
scope = cw.scope()
target = cw.target(scope, cw.targets.CW305, bsfile=bitstream_path)

(ChipWhisperer Target WARNING|File CW305.py:591) Using default Verilog defines (c:\Users\jhaaa\Desktop\cw305_research\scripts\my_venv\Lib\site-packages\chipwhisperer/hardware/firmware/cw305/cw305_aes_defines.v); if this is not what you want, provide them via the defines_files argument


In [3]:
# This returns True if the FPGA 'DONE' pin is high
print(f"FPGA Programmed: {target.is_programmed()}")
# The reference design stores the build timestamp in a register
try:
    print(f"FPGA Build Time: {target.get_fpga_buildtime()}")
except:
    print("Could not read build time. Check if your design includes the standard registers.")

FPGA Programmed: True
FPGA Build Time: 2/2/2026, 00:58


In [4]:
target.vccint_set(1.0) #set VCC-INT to 1V
target.pll.pll_outfreq_set(10e6, 1)

In [5]:
# Configure the Scope for capture
scope.gain.db = 40        # Adjust based on signal strength
scope.adc.samples = 5000  # Number of points to capture
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.io.trigger_source = "tio4"
scope.clock.adc_src = "extclk_x4"

(ChipWhisperer Other ERROR|File util.py:419) Setting unknown attribute trigger_source in <class 'chipwhisperer.capture.scopes.cwhardware.ChipWhispererExtra.GPIOSettings'>


In [12]:
target.fpga_write(0x0c, [0x01]) # Enable ROs

In [7]:
target.fpga_write(0x0c, [0x00]) # Disable ROs

In [8]:
# 1. Input as continuous hex strings
key_hex = "000102030405060708090a0b0c0d0e0f"
pt_hex  = "00112233445566778899aabbccddeeff"

# 2. Convert to list of integers
# bytes.fromhex() creates a bytes object; list() turns it into [0x00, 0x01, ...]
key = list(bytes.fromhex(key_hex))[::-1]
plaintext = list(bytes.fromhex(pt_hex))[::-1]

In [9]:
# 2. Write the Key to REG_CRYPT_KEY ('h0a)
target.fpga_write(0x0a, key)

# 3. Write the Plaintext to REG_CRYPT_TEXTIN ('h06)
target.fpga_write(0x06, plaintext)

# 4. Read the registers back to see values set
key_sent = target.fpga_read(0x0a, 16)
formatted_hex_key = key_sent.hex(' ')
print(f"Key: {formatted_hex_key}")
plaintext_sent = target.fpga_read(0x06, 16)
formatted_hex_plaintext = plaintext_sent.hex(' ')
print(f"Plaintext: {formatted_hex_plaintext}")

Key: 0f 0e 0d 0c 0b 0a 09 08 07 06 05 04 03 02 01 00
Plaintext: ff ee dd cc bb aa 99 88 77 66 55 44 33 22 11 00


In [10]:
# 5. Trigger the encryption by writing to REG_CRYPT_GO ('h05)
target.fpga_write(0x05, [0x01])

In [11]:
# 6. Read the result from REG_CRYPT_CIPHEROUT ('h09)
ciphertext = target.fpga_read(0x09, 16)
formatted_hex = ciphertext.hex(' ')
print(f"Ciphertext: {formatted_hex}")

Ciphertext: 5a c5 b4 70 80 b7 cd d8 30 04 7b 6a d8 e0 c4 69
